<a href="https://colab.research.google.com/github/farzad-mos/House_price_pred/blob/main/price_pred_house.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# House price prediction according to the house features
A	simple	yet	challenging	project,	to	predict	the	housing	price	based	on	certain	factors	like	house	area, bedrooms,	furnished,	nearness	to	mainroad,	etc.	The	dataset	is	small	yet,	it's	complexity	arises	due	to the	fact	that	it	has	strong	multicollinearity.

## 1-Import libreires and datasets

In [ ]:
from	pandas	import	read_csv,	DataFrame,	concat
from	matplotlib.pyplot	import	show,	suptitle,	subplots_adjust,	tight_layout,	subplots,scatter
from	matplotlib.pyplot	import	figure,	title,	xlabel,	ylabel,	grid,	xticks,	tight_layout
from	numpy	import	nan,log,inf
from	seaborn	import	kdeplot,	heatmap,	histplot,	boxplot,	countplot, scatterplot,kdeplot,lmplot,lineplot,violinplot,
from	sklearn.model_selection	import	train_test_split
from	sklearn.linear_model	import	LinearRegression,	Ridge
from	xgboost	import	XGBRegressor
from	sklearn.metrics	import	r2_score,	mean_squared_error,	mean_absolute_error
from	math	import	sqrt
from	pickle	import	dump
from	sklearn.preprocessing	import	LabelEncoder label	=	LabelEncoder()

import	warnings

warnings.filterwarnings("ignore",	category=DeprecationWarning)

# import datasets
df_train	=	read_csv("train.csv")

## 2- Exploratory	Data	Analysis	(EDA)
Initial	Data	Understandings:
* Data	loading	and	Inspection
* Data	Types
* Missing	Values
* Duplicates
* Check basic statistics;
  * Summary	Statistical:	`describe()`
  * counts: `value_counts()`
* Check data types; `select_dtypes()`
* Stats preview plots (Histogram: `hist`, KDE: `kdeplot`)


In [ ]:
# check the data outline
df_train.head()
# check data size
print(f'shape_train:	{df_train.shape}')
# check data info (if there is nan values and missing data)
df_train.info()
# check the columns name
df_train.columns

# find missing data
for	i	in	df_train.columns:
  print(i,"	:	",df_train[i].isnull().sum()) #check missing values

# check duplicated data
df_train.duplicated().sum()

# basic stats
df_train.describe().T

# data type
df_train.select_dtypes(include='object').describe()

# plot hist of columns
df_train.drop('Id',axis=1).hist(bins=15,	figsize=(20,	16),	color='red',
                                edgecolor='black')
suptitle('Histograms	of	Columns',	fontsize=16)
subplots_adjust(hspace=0.5)
show()

# value counts
df_train['YearRemodAdd'].value_counts()

# unique years
df_train['YearRemodAdd'].unique()

# plot KDE
figure(figsize=(12,	4))
kdeplot(df_train['SalePrice'],	fill=True,	color='blue',	alpha=0.6)
title(f'Kernel	Density	Estimate(KDE)	-	Sale	Price', fontsize=16)
xlabel('Sale	Price',	fontsize=12)
ylabel('Density',	fontsize=12)
grid(True,	linestyle='--',	alpha=0.7)
tight_layout()
show()


## 3- Feature engineering
  * feature correlation by heatmap plot (Heatmap: `heatmap`)
  * extract high corrolated features


In [ ]:
# Feature engineering

# check corrolation between features by heatmap
figure(figsize=(35,	20))
heatmap(df_train.select_dtypes(include='number').corr(),	annot=True)
show()

#	function	to	extract	high	positive	correlated	columns
def	get_high_corr(df_train,	threshold):
  corr	=	df_train.select_dtypes('number').corr()
  high_corr	=	corr[(corr	>=	threshold)	&	(corr	!=	1.000)]
  high_corr	=	high_corr.dropna(how='all',	axis=1).dropna(how='all',	axis=0)
  return	high_corr
  #	get	high	positive	correlated	columns
  high_corr	=	get_high_corr(df_train,	0.7)
  #	plot	high	positive	correlated	columns
  figure(figsize=(15,	5))
  plot	=	heatmap(high_corr,	annot=True,	cmap='coolwarm')
  plot.set_title('High	Positive	Correlation')
  show()

#	function	to	extract	high	positive	correlated	columns
def	get_high_corr(df_train,	threshold):
  corr	=	df_train.select_dtypes('number').corr()
  high_corr	=	corr[(corr	<=	threshold)	&	(corr	!=	1.000)]
  high_corr	=	high_corr.dropna(how='all',	axis=1).dropna(how='all',	axis=0)
  return	high_corr
  #	get	high	positive	correlated	columns
  high_corr	=	get_high_corr(df_train,	-0.38)
  #	plot	high	positive	correlated	columns
  figure(figsize=(14,	4))
  plot	=	heatmap(high_corr,	annot=True,	cmap='coolwarm')
  plot.set_title('High	Positive	Correlation')
  show()

## 4- Data Cleaning and Data Relations

In [ ]:
# handling missing values and duplications
for	i	in	df_train.columns:
  print(i,"	:	",df_train[i].isnull().sum())

  #	sum	of	nulls	>=	30
  for	all	datafor	column	in	df_train.columns:
    SumOfNulls	=	df_train[column].isnull().sum()
    if	SumOfNulls	>	30:
      print(f'{column}	:	{SumOfNulls}')


df_train	=	df_train.drop(['Id','EnclosedPorch','BsmtHalfBath','LowQualFinSF',
  'MiscFeature','Fence','PoolQC','Fireplac' ,'3SsnPorch','ScreenPorch',
  'PoolArea','MiscVal','MiscVal','BsmtFinSF2','Fireplaces'],axis=1)
df_train	=	df_train.dropna()
df_train.shape

df_train.head()
df_train['WoodDeckSF'].value_counts()
df_train['OpenPorchSF'].value_counts()
df_train['MasVnrArea'].value_counts()


#	transform	from	flaot	to	int
df_train['MasVnrArea']	=	df_train['MasVnrArea'].astype(int)
df_train['MasVnrArea'].value_counts()
df_train['WoodDeckSF'].mean()
df_train['MasVnrArea'].mean()

#	replace	0	wirh	'nan'	for	train	data
df_train['WoodDeckSF']	=	df_train['WoodDeckSF'].replace(0,nan)
df_train['OpenPorchSF']	=	df_train['OpenPorchSF'].replace(0,nan)
df_train['MasVnrArea']	=	df_train['MasVnrArea'].replace(0,nan)

#	Replace	'nan'	with	mean
mean_WoodDeckSF	=	df_train['WoodDeckSF'].mean()
df_train['WoodDeckSF'].fillna(mean_WoodDeckSF,inplace=True)
mean_OpenPorchSF	=	df_train['OpenPorchSF'].mean()
df_train['OpenPorchSF'].fillna(mean_OpenPorchSF,inplace=True)
mean_MasVnrArea	=	df_train['MasVnrArea'].mean()
df_train['MasVnrArea'].fillna(mean_MasVnrArea,inplace=True)

# Outlier detection and removal
df_train['SalePrice'].describe()

fig.axes	=	subplots(nrows=1,ncols=1)
fig.set_size_inches(8,	6)
boxplot(data=df_train,y="SalePrice",orient="v",ax=axes)
axes.set(ylabel='SalePrice',title="Box	Plot	On	Saleprice")
show()

# Distribution
figure(figsize=(14,	6))
histplot(df_train['SalePrice'],	bins=50,	kde=True,	color='red')
title('Distribution	of	Sale	Price')
xlabel('Sale	Price')
ylabel('count')
show()
print(f"\nSkewness	of	Price:	{df_train['SalePrice'].skew():.2f}")
print(f"Kurtosis	of	Price:	{df_train['SalePrice'].kurt():.2f}")

# Scattering
figure(figsize=(14,	6))
scatterplot(x='LotArea',	y='SalePrice',	data=df_train)
title('Relationship	between	Sale	Price	and	Area')
xlabel('Area')
ylabel('Sale	Price')
show()

# Data relation (Jointplot)
figure(figsize=(14,	4))
jointplot(x='LotArea',	y='SalePrice',	data=df_train,	kind='reg',	height=8,	scatter_kws={'alpha':0.3})
suptitle('Relationship	between	Sale	price	and	Area',	y=1.02,	fontsize=16) s
how()

# trend detection
figure(figsize=(14,	6))
regplot(x='YearBuilt',	y='SalePrice',	data=df_train,	scatter_kws={'alpha':0.3},	line_kws={'color':'red'})
title('Relationship	between	year	built	and	sale	price',	fontsize=16)
xlabel('Year	Built',	fontsize=12)
ylabel('Sale	Price',	fontsize=12)
show()

# Feature (no. of rooms) distribution
figure(figsize=(14,	5))
countplot(x='TotRmsAbvGrd',	data=df_train,	palette='viridis')
title('Distribution	of	number	of	rooms',	fontsize=16)
xlabel('Number	of	rooms',	fontsize=12)
ylabel('Count',	fontsize=12)
show()

# Feature Density	(between	number	of	rooms	and	sale	price)
jointplot(x='TotRmsAbvGrd',	y='SalePrice',	data=df_train,	kind='kde',	height=8,	cmap='viridis')
suptitle('Density	between	number	of	rooms	and	sale	price',	y=1.02,	fontsize=16)
show()

# Feature distribution
 #1- house style
figure(figsize=(14,	5))
countplot(x='HouseStyle',	data=df_train,	palette='viridis')
title('Distribution	of	house	style',	fontsize=16)
xlabel('House	style',	fontsize=12)
ylabel('Count',	fontsize=12)
show()

 #2- type	of	building
figure(figsize=(14,	5))
df_train['BldgType'].value_counts().plot(kind='bar',	color=['green','yellow','blue','red','orange'])
title('Number	of	type	of	building',	fontsize=16)
xlabel('Type	of	building',	fontsize=12)
ylabel('Count',	fontsize=12)
xticks(rotation=45)
show()


 #3- Type of building vs. Sale price
figure(figsize=(14,	6))
violinplot(x='BldgType',	y='SalePrice',	data=df_train,	palette=['green','yellow','blue','red','orange'])
title('Type	of	building	by	Sale	price',	fontsize=16)
xlabel('Type	of	building',	fontsize=12)
ylabel('Sale	price',	fontsize=12)
show()

 #4- Log sale price
figure(figsize=(14,	6))
histplot(log(df_train['SalePrice']).replace(-inf,1e-6),	bins=50,	kde=True,	color='green')
title('Distribution	of	log	Sale	Price')
xlabel('Log	Sale	Price')
ylabel('count')
show()
print(f"\nSkewness	of	Price:	{log(df_train['SalePrice']).replace(-inf,1e-6).skew():.2f}")

# Feature trends (sale/yr.)
avg_price_by_year	=	df_train.groupby('YearBuilt')['SalePrice'].mean()
figure(figsize=(14,	6))
lineplot(x=avg_price_by_year.index,	y=avg_price_by_year.values,	marker='o',	color='purple')
title('Avg	of	sale	price	by	year	built',	fontsize=16)
xlabel('Year	built',	fontsize=12)
ylabel('Avg	of	sale	price',	fontsize=12)
grid(True)
show()

# Relationship between area and sale price by No. room
figure(figsize=(12,	8))
lmplot(x='LotArea',	y='log_SalePrice',	hue='TotRmsAbvGrd',	data=df_train,
       col='TotRmsAbvGrd',	col_wrap=3,	height=4,	aspect=1.2,
       scatter_kws={'alpha':0.4})
suptitle('Relationship	between	Area	and	sale	price	by	number	of	rooms',
         y=1.05,	fontsize=18)
show()

## 5- Data Transformation & Spliting

In [ ]:
# check data type to transform from str. to int.
train_obj	=	df_train.select_dtypes(include='object')
train_obj.shape
train_non_obj	=	df_train.select_dtypes(exclude='object')
train_non_obj.shape


for	col	in	train_obj.columns:
  print(f'{col}	:	{train_obj[col].nunique()}')


for col	in	train_obj.columns:
    nuniques	=	train_obj[col].nunique()
    if	nuniques	==	3:
      print(f'{col}	:	{train_obj[col].nunique()}')


#### Transformations:

 #	LandSlope:	Slope	of	property
 #	Gtl	(Gentle)
 #	Mod	(Moderate)
 #	Sev	(Severe)		t
 rain_obj['LandSlope'].value_counts()	#	ordinal

 # pi plot of LandShape
 train_obj['LandSlope'].value_counts().plot.pie(autopct='%0.2f%%')
 show()

 # Transform data type: change landshape to no.
 train_obj['LandSlope']=	train_obj['LandSlope'].replace('Gtl',1)
 train_obj['LandSlope']=	train_obj['LandSlope'].replace('Mod',2)
 train_obj['LandSlope']=	train_obj['LandSlope'].replace('Sev',3)
 train_obj['LandSlope'].value_counts() #check number of data per group


 #	GarageFinish:	Interior	finish	of	the	garage
 #	Unf	(Unfinished)
 #	RFn	(Rough	Finished)
 #	Fin	(Finished)
 train_obj['GarageFinish'].value_counts()	#	ordinal

 # pi plot of GarageFinish
 train_obj['GarageFinish'].value_counts().plot.pie(autopct='%0.2f%%') show()

 # Transform data type: change GarageFinish to no.
 train_obj['GarageFinish']=	train_obj['GarageFinish'].replace('Unf',1)
 train_obj['GarageFinish']=	train_obj['GarageFinish'].replace('RFn',2)
 train_obj['GarageFinish']=	train_obj['GarageFinish'].replace('Fin',3)
 train_obj['GarageFinish'].value_counts() #check number of data per group

 #	PavedDrive:	Paved	driveway
 #	Y	(Yes	Paved)
 #	P	(Partially	Paved)
 #	N	(Not	Paved)
 train_obj['PavedDrive'].value_counts()	#	ordinal

 # pi plot of PaveDrive
 train_obj['PavedDrive'].value_counts().plot.pie(autopct='%0.2f%%')
 show()

 # Transform data type: change Pavedrive to No.
train_obj['PavedDrive']=	train_obj['PavedDrive'].replace('Y',1)
train_obj['PavedDrive']=	train_obj['PavedDrive'].replace('N',2)
train_obj['PavedDrive']=	train_obj['PavedDrive'].replace('P',3)
train_obj['PavedDrive'].value_counts() #check number of data per group


#	LotShape	:	General	shape	of	property
#	Reg	(Regular)
#	IR1	(Slightly	irregular)
#	IR2	(Moderately	irregular)
#	IR3	(Irregular)
train_obj['LotShape'].value_counts()	#	ordinal

# pi plot of LotShape
train_obj['LotShape'].value_counts().plot.pie(autopct='%0.2f%%')
show()

# Transform data
train_obj['LotShape']=	train_obj['LotShape'].replace('Reg',1)
train_obj['LotShape']=	train_obj['LotShape'].replace('IR1',2)
train_obj['LotShape']=	train_obj['LotShape'].replace('IR2',3)
train_obj['LotShape']=	train_obj['LotShape'].replace('IR3',4)
train_obj['LotShape'].value_counts()

#	LandContour	:	Flatness	of	the	property
#	Lvl	(Level)
#	Bnk	(Banked	–	Slope	from	the	street)
#	HLS	(Hillside	–	Significant	slope)
#	Low	(Depression	–	Below	street	level)
train_obj['LandContour'].value_counts()	#	ordinal

train_obj['LandContour'].value_counts().plot.pie(autopct='%0.2f%%')
show()

train_obj['LandContour']=	train_obj['LandContour'].replace('Lvl',1)
train_obj['LandContour']=	train_obj['LandContour'].replace('Bnk',2)
train_obj['LandContour']=	train_obj['LandContour'].replace('HLS',3)
train_obj['LandContour']=	train_obj['LandContour'].replace('Low',4)

train_obj['LandContour'].value_counts()

#	ExterQual:	Exterior	material	quality
#	Fa	(Fair)
#	TA	(Typical/Average)
#	Gd	(Good)
#	Ex	(Excellent)
train_obj['ExterQual'].value_counts()	#	ordinal

train_obj['ExterQual'].value_counts().plot.pie(autopct='%0.2f%%')
show()

train_obj['ExterQual']=	train_obj['ExterQual'].replace('Fa',1)
train_obj['ExterQual']=	train_obj['ExterQual'].replace('TA',2)
train_obj['ExterQual']=	train_obj['ExterQual'].replace('Gd',3)
train_obj['ExterQual']=	train_obj['ExterQual'].replace('Ex',4)
train_obj['ExterQual'].value_counts()

#	ExterCond:	Present	condition	of	the	material	on	the	exterior
#	Fa	(Fair)
#	TA	(Typical/Average)
#	Gd	(Good)
#	Ex	(Excellent)
train_obj['ExterCond'].value_counts()	#	ordinal

countplot(x='ExterCond',	data=df_train)
show()

train_obj['ExterCond']=	train_obj['ExterCond'].replace('Fa',1)
train_obj['ExterCond']=	train_obj['ExterCond'].replace('TA',2)
train_obj['ExterCond']=	train_obj['ExterCond'].replace('Gd',3)
train_obj['ExterCond']=	train_obj['ExterCond'].replace('Ex',4)
train_obj['ExterCond'].value_counts()

#	BsmtCond	:	General	condition	of	the	basement
#	Gd	(Good)
#	TA	(Typical/Average)
#	Fa	(Fair)
#	Po	(Poor)
train_obj['BsmtCond'].value_counts()	#	ordinal

train_obj['BsmtCond'].value_counts().plot.pie(autopct='%0.2f%%')
show()

train_obj['BsmtCond']=	train_obj['BsmtCond'].replace('Gd',1)
train_obj['BsmtCond']=	train_obj['BsmtCond'].replace('TA',2)
train_obj['BsmtCond']=	train_obj['BsmtCond'].replace('Fa',3)
train_obj['BsmtCond']=	train_obj['BsmtCond'].replace('Po',4)
train_obj['BsmtCond'].value_counts()


#	BsmtQual:	Height	of	the	basement
#	Fa	(Fair)
#	TA	(Typical/Average)
#	Gd	(Good)
#	Ex	(Excellent)
train_obj['BsmtQual'].value_counts()	#	ordinal

train_obj['BsmtQual'].value_counts().plot.pie(autopct='%0.2f%%')
show()

train_obj['BsmtQual']=	train_obj['BsmtQual'].replace('Fa',1)
train_obj['BsmtQual']=	train_obj['BsmtQual'].replace('TA',2)
train_obj['BsmtQual']=	train_obj['BsmtQual'].replace('Gd',3)
train_obj['BsmtQual']=	train_obj['BsmtQual'].replace('Ex',4)

train_obj['BsmtQual'].value_counts()


#	BsmtExposure	:	Walkout	or	garden	level	basement	walls
#	No
#	Mn	(Minimum)
#	Av	(Average)
#	Gd	(Good)
train_obj['BsmtExposure'].value_counts()	#	ordinal

train_obj['BsmtExposure'].value_counts().plot.pie(autopct='%0.2f%%')
show()

train_obj['BsmtExposure']=	train_obj['BsmtExposure'].replace('No',1)
train_obj['BsmtExposure']=	train_obj['BsmtExposure'].replace('Mn',2)
train_obj['BsmtExposure']=	train_obj['BsmtExposure'].replace('Av',3)
train_obj['BsmtExposure']=	train_obj['BsmtExposure'].replace('Gd',4)
train_obj['BsmtExposure'].value_counts()


# check transformed data
train_data	=	concat([train_non_obj,train_obj],axis=1)
train_data.info()


train_obj2	=	train_data.select_dtypes(include='object')
train_non_obj2	=	train_data.select_dtypes(exclude='object')


# check label Encoder

for	i	in	range(0,train_obj2.shape[1]):
  train_obj2.iloc[:,i]=label.fit_transform(train_obj2.iloc[:,i])


train_obj2	=	train_obj2.astype(int)
train_obj2.info()

# final transformed & encoded data
final_train_data	=	concat([train_non_obj2,train_obj2],axis=1)

final_train_data['LotShape'].value_counts()
final_train_data['KitchenQual'].value_counts()


### Data spliting
X=	final_train_data.drop(['SalePrice'],axis=1)
y=	final_train_data['SalePrice']
x_train,x_test,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=1234)

print(x_train.shape,	x_test.shape)
print(y_train.shape,	y_test.shape)

## 6- Model Definition, Training & Comparision
3 models have been defined:
* Ridge
* Linear Regression
* XGBRegression

In [ ]:
r_2=[]
mse	=[]
rmse=[]
mae=[]
def	reg(model):
  model.fit(x_train,y_train)
  pred	=	model.predict(x_test)
  R2	=	r2_score(y_test,pred)
  MSE	=	mean_squared_error(y_test,pred)
  RMSE	=	sqrt(mean_squared_error(y_test,pred))
  MAE	=	mean_absolute_error(y_test,pred)
  r_2.append(R2)
  mse.append(MSE)
  rmse.append(RMSE)
  mae.append(MAE)

# models
Ridge_model	=	Ridge()
LinearRegression_model	=	LinearRegression()
XGBRegressor_model	=	XGBRegressor()

Algorithms	=	['LinearRegression','Ridge','XGBRegressor']


reg(LinearRegression_model)
reg(Ridge_model)
reg(XGBRegressor_model)

# fitting results comparision
result	=	DataFrame({'Algorithms':Algorithms,'R2':r_2,'mse':mse,'rmse':rmse,'mae':mae})


### Model Comparision:

* XGBRegressor	stands	out	as	the	best-performing	algorithm	among	the	three:
 * It	has	the	highest	R2	value	(0.88448),	indicating	that	it	explains	a	significantly	larger	proportion	of	the	variance	in	the	target	variable compared	to	Linear	Regression	and	Ridge.
 * It	has	the	lowest	mse	(811,541,568.0),	rmse	(28,487.57),	and	mae	(18,410.43),	which	means	its	predictions	have	the	smallest average	errors.
* Linear	Regression	and	Ridge	perform	very	similarly:
 * Their	R2	values	are	almost	identical	(around	0.847).
 * Their	mse,	rmse,	and	mae	values	are	also	very	close,	indicating	similar	levels	of	prediction	error.	Ridge's	mae	is	slightly	lower	than Linear	Regression's,	but	the	difference	is	minimal.

In [ ]:
# comparision visualization
XGBRegressor_model.fit(x_train,y_train)
pred	=	XGBRegressor_model.predict(x_test)
scatter(y_test,	pred,	color='g')
show()

Ridge_model.fit(x_train,y_train)
pred	=	Ridge_model.predict(x_test)
scatter(y_test,	pred,	color='g')
show()

LinearRegression_model.fit(x_train,y_train)
pred	=	LinearRegression_model.predict(x_test)
scatter(y_test,	pred,	color='g')
show()

## 7- Conclusion
Based	on	these	evaluation	metrics,	the	XGBRegressor	is	the	superior	model	for	this	regression	task.	It demonstrates	significantly	better	predictive	power	by	explaining	more	variance	and	achieving	lower error	rates	compared	to	both	Linear	Regression	and	Ridge.	This	suggests	that	the	underlying	data	might have	non-linear	relationships	that	XGBRegressor	is	better	equipped	to	capture.